In [ ]:
import pandas as pd
import numpy as np
import os
from time import time
from sklearn.model_selection import train_test_split
from pydts.examples_utils.generate_simulations_data import generate_quick_start_df
from pydts.examples_utils.plots import plot_example_pred_output
from pydts.examples_utils.plots import add_panel_text
from pydts.fitters import TwoStagesFitter, DataExpansionFitter

from pydts.data_generation import EventTimesSampler
from matplotlib import pyplot as plt
import warnings
import pickle
from copy import deepcopy
from sklearn.model_selection import KFold
pd.set_option("display.max_rows", 500)
warnings.filterwarnings('ignore')
%matplotlib inline
slicer = pd.IndexSlice

In [ ]:
OUTPUT_DIR = '/home/tomer.me/DiscreteTimeSurvivalPenalization/output'

# Sampling data

In [ ]:
n_cov = 100
beta1 = np.zeros(n_cov)
beta1[:5] = (-np.log([1.2, 0.9, 1.2, 0.8, 1.2]))
beta2 = np.zeros(n_cov)
beta2[:5] = (-np.log([1.4, 0.8, 1.1, 1, 1.1]))

real_coef_dict = {
    "alpha": {
        1: lambda t: -2.6 + 0.4 * np.log(t),
        2: lambda t: -2.7 + 0.5 * np.log(t)
    },
    "beta": {
        1: beta1,
        2: beta2
    }
}

n_patients = 15000
d_times = 15
j_events = 2

ets = EventTimesSampler(d_times=d_times, j_event_types=j_events)

seed = 0
means_vector = np.zeros(n_cov)
covariance_matrix = 1 * np.identity(n_cov)
clip_value = 1.8

covariates = [f'Z{i + 1}' for i in range(n_cov)]

patients_df = pd.DataFrame(data=pd.DataFrame(data=np.random.multivariate_normal(means_vector, covariance_matrix,
                                                                                size=n_patients),
                                             columns=covariates))
patients_df.clip(lower=-1 * clip_value, upper=clip_value, inplace=True)
patients_df = ets.sample_event_times(patients_df, hazard_coefs=real_coef_dict, seed=seed)
patients_df = ets.sample_independent_lof_censoring(patients_df, prob_los_at_t=0.01 * np.ones_like(ets.times),
                                                   seed=seed + 1)
patients_df = ets.update_event_or_lof(patients_df)

patients_df.index.name = 'pid'
patients_df = patients_df.reset_index()

In [ ]:
from pydts.examples_utils.plots import plot_events_occurrence
plot_events_occurrence(patients_df)

In [ ]:
penalizers = [np.exp(v) for v in np.arange(-9, 0.25, step=0.25)] 
models = {}
n_splits = 5

for idp, penalizer in enumerate(penalizers):

    kfold_cv = KFold(n_splits=n_splits, shuffle=True, random_state=(100+idp))
    
    folds_models = []
    
    for i_fold, (train_index, test_index) in enumerate(kfold_cv.split(patients_df)):
        train_df, test_df = patients_df.loc[train_index], patients_df.loc[test_index]
    
        L1_regularized_fitter = TwoStagesFitter()

        fit_beta_kwargs = {
            'model_kwargs': {
                'penalizer': penalizer,
                'l1_ratio': 1
            }
        }

        print(f'Start fitting with L1 regularization coef: {penalizer}, fold {i_fold}')
        start = time()
        L1_regularized_fitter.fit(df=train_df.drop(['C', 'T'], axis=1), fit_beta_kwargs=fit_beta_kwargs, nb_workers=1)
        end = time()
        print(f'Finished: {end - start} sec')
        
        folds_models.append(deepcopy(L1_regularized_fitter))

    models[penalizer] = folds_models

In [ ]:
with open(os.path.join(OUTPUT_DIR, f'rerun_regularization_models.pkl'), "wb") as fp: 
    pickle.dump(models, fp)

In [ ]:
p_threshold = 0.05
fontsize = 13

mode = 'll' # or 'score'

if mode == 'score':
    from pydts.utils import get_expanded_df
    exp_patients_df = get_expanded_df(patients_df)
    exp_patients_df['X_copy'] = np.ones_like(exp_patients_df['X'])

fig, axes = plt.subplots(2, 2, figsize=(11,6), gridspec_kw={'width_ratios': [3, 1]})

for idp, penalizer in enumerate(penalizers):
    likelihood_1 = []
    likelihood_2 = []
    significant_coef_1 = []
    significant_coef_2 = []
    cv_diff_1 = []
    cv_diff_2 = []
    
    for i_fold in range(n_splits):
        likelihood_1.append(models[penalizer][i_fold].beta_models[1].log_likelihood_)
        tmp = models[penalizer][i_fold].beta_models[1].summary[['coef', 'p']]
        significant_coef_1.append(len(tmp[tmp['p'] <= p_threshold]))
        
        likelihood_2.append(models[penalizer][i_fold].beta_models[2].log_likelihood_)
        tmp = models[penalizer][i_fold].beta_models[2].summary[['coef', 'p']]
        significant_coef_2.append(len(tmp[tmp['p'] <= p_threshold]))
        
        if mode == 'score':
            cv_diff_1.append(
                models[penalizer][i_fold].beta_models[1].score(exp_patients_df) - \
                    models[penalizer][i_fold].beta_models[1].log_likelihood_
            )

            cv_diff_2.append(
                models[penalizer][i_fold].beta_models[2].score(exp_patients_df) - \
                    models[penalizer][i_fold].beta_models[2].log_likelihood_
            )
        
    mean_likelihood_1 = np.mean(np.array(likelihood_1))
    err_1 = np.std(np.array(likelihood_1))
    mean_sign_1 = np.median(np.array(significant_coef_1))
    
    if mode == 'score':
        sum_1 = np.sum(np.array(cv_diff_1))
    
    mean_likelihood_2 = np.mean(np.array(likelihood_2))
    err_2 = np.std(np.array(likelihood_2))
    mean_sign_2 = np.median(np.array(significant_coef_2))
    
    if mode == 'score':
        sum_2 = np.sum(np.array(cv_diff_2))

    ax = axes[0, 0]
    add_panel_text(ax, 'a', fsz=fontsize)
    if mode == 'score':
        ax.scatter(np.log(penalizer), sum_1, color='g', alpha=0.8)
    else:
        ax.errorbar(np.log(penalizer), mean_likelihood_1, yerr=err_1, fmt="o", color='g', alpha=0.5)
        
    ax.set_xlabel(r'log($\lambda$)', fontsize=fontsize)
    ax.set_ylabel('Log-Partial Likelihood', fontsize=fontsize, color='g')
    ax.set_xticks([t for t in range(-9,1,1)])
    ax.set_xticklabels([f'{t}' for t in range(-9,1,1)])    
    #ax.set_title(r'Stratified CoxPH for $\beta_1$ Estimation', fontsize=fontsize)
    ax.set_title(r'$\beta_1$', fontsize=fontsize)
    
    chosen_lambda = -6.0

    ax.tick_params(axis='y', colors='g')
    ax.axvline(chosen_lambda, color='tab:blue', alpha=0.1, ls='--', lw=1)

    
    if idp == 0:
        #ax.legend()
        ax2 = ax.twinx()
        ax2.set_ylabel('5-Folds Median \n Number of Coefficients', fontsize=fontsize)
        ax2.set_ylim([-0.3, 10])

    ax2.scatter(np.log(penalizer), mean_sign_1, color='k', alpha=0.8, marker='P')
    ax2.tick_params(axis='y', colors='k')

    ax = axes[1, 0]
    add_panel_text(ax, 'b', fsz=fontsize)

    if mode == 'score':
        ax.scatter(np.log(penalizer), sum_2, color='g', alpha=0.8, label='Mean')
    else:
        ax.errorbar(np.log(penalizer), mean_likelihood_2, yerr=err_2, fmt="o", color='g', alpha=0.5)

    ax.set_xlabel(r'log($\lambda$)', fontsize=fontsize)
    ax.set_ylabel('Log-Partial Likelihood', fontsize=fontsize, color='g')
    ax.set_xticks([t for t in range(-9,1,1)])
    ax.set_xticklabels([f'{t}' for t in range(-9,1,1)])
    #ax.set_title(r'Stratified CoxPH for $\beta_2$ Estimation', fontsize=fontsize)
    ax.set_title(r'$\beta_2$', fontsize=fontsize)

    ax.axvline(chosen_lambda, color='tab:blue', alpha=0.1, ls='--', lw=1)
    ax.tick_params(axis='y', colors='g')

    if idp == 0:
        #ax.legend()
        ax3 = ax.twinx()
        ax3.set_ylabel('5-Folds Median \n Number of Coefficients', fontsize=fontsize)
        ax3.set_ylim([-0.3, 10])
    
    ax3.scatter(np.log(penalizer), mean_sign_2, color='k', alpha=0.8, marker='P')
    
for idp, penalizer in enumerate(penalizers):
    tmp_j1_params_df = pd.DataFrame()
    tmp_j2_params_df = pd.DataFrame()

    for i_fold in range(n_splits):
        tmp_j1_params_df = pd.concat([tmp_j1_params_df, models[penalizer][i_fold].beta_models[1].params_], axis=1)
        tmp_j2_params_df = pd.concat([tmp_j2_params_df, models[penalizer][i_fold].beta_models[2].params_], axis=1)
    
    ser_1 = tmp_j1_params_df.mean(axis=1) 
    ser_1.name = penalizer
    ser_2 = tmp_j2_params_df.mean(axis=1) 
    ser_2.name = penalizer    
    
    if idp == 0:
        j1_params_df = ser_1.to_frame()
        j2_params_df = ser_2.to_frame()
    else:
        j1_params_df = pd.concat([j1_params_df, ser_1], axis=1)
        j2_params_df = pd.concat([j2_params_df, ser_2], axis=1)
        
        
for i in range(n_cov):
    ax = axes[0, 1]
    add_panel_text(ax, 'c', fsz=fontsize)

    ax.plot(penalizers, j1_params_df.iloc[i].values, label=f'Z{i}', lw=1)

    if i == 0:
        ax.set_xlim([0, 0.04])
        ax.set_ylabel('5-Fold Mean Coefficient', fontsize=fontsize, color='brown')
        ax.set_xlabel(r'$\lambda$', fontsize=fontsize)
        ax.set_title(r'$\beta_1$', fontsize=fontsize)
        ax.axvline(np.exp(chosen_lambda), color='tab:blue', alpha=1, ls='--', lw=1)
    
    ax.tick_params(axis='y', colors='brown')
    
    ax = axes[1, 1]
    add_panel_text(ax, 'd', fsz=fontsize)

    ax.plot(penalizers, j2_params_df.iloc[i].values, label=f'Z{i}', lw=1)

    if i == 0:
        ax.set_xlim([0, 0.04])
        ax.set_ylabel('5-Fold Mean Coefficient', fontsize=fontsize, color='brown')
        ax.set_xlabel(r'$\lambda$', fontsize=fontsize)
        ax.set_title(r'$\beta_2$', fontsize=fontsize)
        ax.axvline(np.exp(chosen_lambda), color='tab:blue', alpha=1, ls='--', lw=1)
    
    ax.tick_params(axis='y', colors='brown')


fig.tight_layout()

fig.savefig(os.path.join(OUTPUT_DIR, 'rerun_regularization_effect.png'), dpi=300)

In [ ]:
penalizer = np.exp(-6.0) 

L1_regularized_fitter = TwoStagesFitter()

fit_beta_kwargs = {
    'model_kwargs': {
        'penalizer': penalizer,
        'l1_ratio': 1
    }
}

print(f'Start fitting with L1 regularization coef: {penalizer}')
start = time()
L1_regularized_fitter.fit(df=patients_df.drop(['C', 'T'], axis=1), fit_beta_kwargs=fit_beta_kwargs, nb_workers=1)
end = time()
print(f'Finished: {end - start} sec')

In [ ]:
L1_regularized_fitter.beta_models[1].summary[['coef', 'p']].round(3)

In [ ]:
event_1 = L1_regularized_fitter.beta_models[1].summary[['coef', 'p']]
event_1 = event_1[event_1['p'] <= p_threshold]
event_1['True'] = (-np.log([1.2, 0.9, 1.2, 0.8, 1.2]))
event_1.columns = ['Estimate', 'P-value', 'True']
event_1

In [ ]:
print(event_1[['True', 'Estimate', 'P-value']].to_latex())

In [ ]:
event_2 = L1_regularized_fitter.beta_models[2].summary[['coef', 'p']]
event_2 = event_2[event_2['p'] <= p_threshold]
event_2['True'] = (-np.log([1.4, 0.8, 1.1, 1.1]))
event_2.columns = ['Estimate', 'P-value', 'True']
event_2

In [ ]:
print(event_2[['True', 'Estimate', 'P-value']].to_latex())

In [ ]:
# Additional case (with correlations)

In [ ]:
n_cov = 100
beta1 = np.zeros(n_cov)
beta1[:5] = (-np.log([1.2, 0.9, 1.2, 0.8, 1.2]))
beta2 = np.zeros(n_cov)
beta2[:5] = (-np.log([1.4, 0.8, 1.1, 1, 1.1]))

real_coef_dict = {
    "alpha": {
        1: lambda t: -2.6 + 0.4 * np.log(t),
        2: lambda t: -2.7 + 0.5 * np.log(t)
    },
    "beta": {
        1: beta1,
        2: beta2
    }
}

n_patients = 15000
d_times = 15
j_events = 2

ets = EventTimesSampler(d_times=d_times, j_event_types=j_events)

seed = 0
means_vector = np.zeros(n_cov)
covariance_matrix = 1 * np.identity(n_cov)
covariance_matrix[0, 8] = 0.4
covariance_matrix[8, 0] = 0.4
covariance_matrix[1, 9] = 0.6
covariance_matrix[9, 1] = 0.6
covariance_matrix[4, 10] = 0.2
covariance_matrix[10, 4] = 0.2

clip_value = 1.8

covariates = [f'Z{i + 1}' for i in range(n_cov)]

patients_df = pd.DataFrame(data=pd.DataFrame(data=np.random.multivariate_normal(means_vector, covariance_matrix,
                                                                                size=n_patients),
                                             columns=covariates))
patients_df.clip(lower=-1 * clip_value, upper=clip_value, inplace=True)
patients_df = ets.sample_event_times(patients_df, hazard_coefs=real_coef_dict, seed=seed)
patients_df = ets.sample_independent_lof_censoring(patients_df, prob_los_at_t=0.01 * np.ones_like(ets.times),
                                                   seed=seed + 1)
patients_df = ets.update_event_or_lof(patients_df)

patients_df.index.name = 'pid'
patients_df = patients_df.reset_index()

In [ ]:
from pydts.examples_utils.plots import plot_events_occurrence
plot_events_occurrence(patients_df)

In [ ]:
penalizers = [np.exp(v) for v in np.arange(-9, -3.25, step=0.25)] 
models = {}
n_splits = 5

for idp, penalizer in enumerate(penalizers):
    
    L1_regularized_fitter = TwoStagesFitter()

    fit_beta_kwargs = {
        'model_kwargs': {
            'penalizer': penalizer,
            'l1_ratio': 1
        }
    }

    print(f'Start fitting with L1 regularization coef: {penalizer}, fold {i_fold}')
    start = time()
    L1_regularized_fitter.fit(df=patients_df.drop(['C', 'T'], axis=1), fit_beta_kwargs=fit_beta_kwargs, nb_workers=1)
    end = time()
    print(f'Finished: {end - start} sec')
        
    models[penalizer] = deepcopy(L1_regularized_fitter)

In [ ]:
with open(os.path.join(OUTPUT_DIR, f"rerun_regularization_models_additional_correlated.pkl"), "wb") as fp: 
    pickle.dump(models, fp)

In [ ]:
p_threshold = 0.05


mode = 'll' # or 'score'

if mode == 'score':
    from pydts.utils import get_expanded_df
    exp_patients_df = get_expanded_df(patients_df)
    exp_patients_df['X_copy'] = np.ones_like(exp_patients_df['X'])

fig, axes = plt.subplots(2, 2, figsize=(11,6), gridspec_kw={'width_ratios': [3, 1]})

for idp, penalizer in enumerate(penalizers):
    likelihood_1 = []
    likelihood_2 = []
    significant_coef_1 = []
    significant_coef_2 = []
    cv_diff_1 = []
    cv_diff_2 = []
    
    likelihood_1.append(models[penalizer].beta_models[1].log_likelihood_)
    tmp = models[penalizer].beta_models[1].summary[['coef', 'p']]
    significant_coef_1.append(len(tmp[tmp['p'] <= p_threshold]))

    likelihood_2.append(models[penalizer].beta_models[2].log_likelihood_)
    tmp = models[penalizer].beta_models[2].summary[['coef', 'p']]
    significant_coef_2.append(len(tmp[tmp['p'] <= p_threshold]))

    if mode == 'score':
        cv_diff_1.append(
            models[penalizer].beta_models[1].score(exp_patients_df) - \
                models[penalizer].beta_models[1].log_likelihood_
        )

        cv_diff_2.append(
            models[penalizer].beta_models[2].score(exp_patients_df) - \
                models[penalizer].beta_models[2].log_likelihood_
        )
        
    mean_likelihood_1 = np.mean(np.array(likelihood_1))
    err_1 = np.std(np.array(likelihood_1))
    mean_sign_1 = np.median(np.array(significant_coef_1))
    
    if mode == 'score':
        sum_1 = np.sum(np.array(cv_diff_1))
    
    mean_likelihood_2 = np.mean(np.array(likelihood_2))
    err_2 = np.std(np.array(likelihood_2))
    mean_sign_2 = np.median(np.array(significant_coef_2))
    
    if mode == 'score':
        sum_2 = np.sum(np.array(cv_diff_2))

    ax = axes[0, 0]
    add_panel_text(ax, 'a', fsz=fontsize)
    if mode == 'score':
        ax.scatter(np.log(penalizer), sum_1, color='g', alpha=0.8)
    else:
        ax.errorbar(np.log(penalizer), mean_likelihood_1, yerr=err_1, fmt="o", color='g', alpha=0.5)
        
    ax.set_xlabel(r'log($\lambda$)', fontsize=fontsize)
    ax.set_ylabel('Log-Partial Likelihood', fontsize=fontsize, color='g')
    ax.set_xticks([t for t in range(-9,1,1)])
    ax.set_xticklabels([f'{t}' for t in range(-9,1,1)])    
    #ax.set_title(r'Stratified CoxPH for $\beta_1$ Estimation', fontsize=fontsize)
    ax.set_title(r'$\beta_1$', fontsize=fontsize)

    
    chosen_lambda = -5.75

    ax.tick_params(axis='y', colors='g')
    ax.axvline(chosen_lambda, color='tab:blue', alpha=0.1, ls='--', lw=1)

    
    if idp == 0:
        #ax.legend()
        ax2 = ax.twinx()
        ax2.set_ylabel('5-Folds Median \n Number of Coefficients', fontsize=fontsize)
        ax2.set_ylim([-0.3, 10])

    ax2.scatter(np.log(penalizer), mean_sign_1, color='k', alpha=0.8, marker='P')
    ax2.tick_params(axis='y', colors='k')

    ax = axes[1, 0]
    add_panel_text(ax, 'b', fsz=fontsize)

    if mode == 'score':
        ax.scatter(np.log(penalizer), sum_2, color='g', alpha=0.8, label='Mean')
    else:
        ax.errorbar(np.log(penalizer), mean_likelihood_2, yerr=err_2, fmt="o", color='g', alpha=0.5)

    ax.set_xlabel(r'log($\lambda$)', fontsize=fontsize)
    ax.set_ylabel('Log-Partial Likelihood', fontsize=fontsize, color='g')
    ax.set_xticks([t for t in range(-9,1,1)])
    ax.set_xticklabels([f'{t}' for t in range(-9,1,1)])
    #ax.set_title(r'Stratified CoxPH for $\beta_2$ Estimation', fontsize=fontsize)
    ax.set_title(r'$\beta_2$', fontsize=fontsize)

    ax.axvline(chosen_lambda, color='tab:blue', alpha=0.1, ls='--', lw=1)
    ax.tick_params(axis='y', colors='g')

    if idp == 0:
        #ax.legend()
        ax3 = ax.twinx()
        ax3.set_ylabel('5-Folds Median \n Number of Coefficients', fontsize=fontsize)
        ax3.set_ylim([-0.3, 10])
    
    ax3.scatter(np.log(penalizer), mean_sign_2, color='k', alpha=0.8, marker='P')
    
for idp, penalizer in enumerate(penalizers):
    
    ser_1 = models[penalizer].beta_models[1].params_
    ser_1.name = penalizer
    ser_2 = models[penalizer].beta_models[2].params_ 
    ser_2.name = penalizer    
    
    if idp == 0:
        j1_params_df = ser_1.to_frame()
        j2_params_df = ser_2.to_frame()
    else:
        j1_params_df = pd.concat([j1_params_df, ser_1], axis=1)
        j2_params_df = pd.concat([j2_params_df, ser_2], axis=1)
        
        
for i in range(n_cov):
    ax = axes[0, 1]
    add_panel_text(ax, 'c', fsz=fontsize)

    ax.plot(penalizers, j1_params_df.iloc[i].values, label=f'Z{i}', lw=1)

    if i == 0:
        ax.set_xlim([0, 0.03])
        ax.set_ylabel('5-Fold Mean Coefficient', fontsize=fontsize, color='brown')
        ax.set_xlabel(r'$\lambda$', fontsize=fontsize)
        ax.set_title(r'$\beta_1$', fontsize=fontsize)
        ax.axvline(np.exp(chosen_lambda), color='tab:blue', alpha=1, ls='--', lw=1)
    
    ax.tick_params(axis='y', colors='brown')
    
    ax = axes[1, 1]
    add_panel_text(ax, 'd', fsz=fontsize)

    ax.plot(penalizers, j2_params_df.iloc[i].values, label=f'Z{i}', lw=1)

    if i == 0:
        ax.set_xlim([0, 0.03])
        ax.set_ylabel('5-Fold Mean Coefficient', fontsize=fontsize, color='brown')
        ax.set_xlabel(r'$\lambda$', fontsize=fontsize)
        ax.set_title(r'$\beta_2$', fontsize=fontsize)
        ax.axvline(np.exp(chosen_lambda), color='tab:blue', alpha=1, ls='--', lw=1)
    
    ax.tick_params(axis='y', colors='brown')

fig.tight_layout()

fig.savefig(os.path.join(OUTPUT_DIR, 'rerun_regularization_effect_additional.png'), dpi=300)

In [ ]:
events = pd.DataFrame()

for idp, penalizer in enumerate(penalizers):
    event_1 = models[penalizer].beta_models[1].summary[['coef', 'p']]
    event_1 = event_1[event_1['p'] <= p_threshold]
    event_1.columns = ['Estimate', 'P-value']

    event_2 = models[penalizer].beta_models[2].summary[['coef', 'p']]
    event_2 = event_2[event_2['p'] <= p_threshold]
    event_2.columns = ['Estimate', 'P-value']
    
    events = pd.concat([events, 
                        pd.concat([pd.concat([event_1, event_2], keys=['1', '2'])], keys=[np.log(penalizer)])])

events.index = events.index.set_names([r'Log($\lambda$)', 'J', 'Covariate'])
events['True'] = np.nan

for idx, row in events.iterrows():
    events.loc[idx, 'True'] = real_coef_dict['beta'][int(idx[1])][int(idx[2].replace('Z', '')) - 1]

In [ ]:
tmp = events.groupby([r'Log($\lambda$)', 'J'])['True'].value_counts() 
fp = tmp.loc[slicer[:, :, 0]].to_frame()
fp.columns = ['False Positives']
fp.unstack('J').astype(int)

In [ ]:
tmp = events.loc[slicer[:, :, ['Z1', 'Z2', 'Z3', 'Z4', 'Z5']]].groupby([r'Log($\lambda$)', 'J'])['Estimate'].count()
tmp = tmp.to_frame()
tmp.columns = ['True Positives']
tp = tmp.copy()
tp.unstack(['J']).fillna(0).astype(int)

In [ ]:
print(pd.concat([fp.unstack('J'), tp.unstack(['J'])], axis=1).fillna(0).astype(int).to_latex(escape=False))